## Predict

이미 훈련된 Faster RCNN 모델을 로드하여 객체 검출을 하는 과정입니다.

In [ ]:
from __future__ import division
import os
import cv2
import numpy as np
import sys
import pickle
import time
from keras_frcnn import config
from keras import backend as K
from keras.layers import Input
from keras.models import Model
from keras_frcnn import roi_helpers
import keras_frcnn.resnet as nn

### 데이터셋 설정

아래는 데이터셋 및 객체 검출 설정에 필요한 환경 변수입니다. 사용하는 데이터셋 및 모델에 맞춰서 변경하시기 바랍니다.

 - `weight_file_path` : 입력 weight 파일의 경로 및 이름을 지정합니다.
 - `config_file_path` : 모델 정보를 담은 pickle 파일의 경로 및 이름을 지정합니다.
 - `predict_source` : 객체 검출을 할 대상 이미지 파일이 있는 디렉토리를 지정합니다.
 - `predict_result` : 객체 검출 결과가 출력될 디렉토리를 지정합니다. 자동으로 해당 디렉토리를 생성하지 않으므로 미리 생성해두셔야 합니다.
 - `bbox_threshold` : 검출된 경계 상자를 양성(positive)로 판정할 임계값입니다.

In [ ]:
weight_file_path = './frcnn.hdf5'
config_file_path = './config.pickle'
predict_source = './images'
predict_result = './results'
bbox_threshold = 0.8

### 모델 빌드

Faster RCNN with ResNet50 모델을 빌드하고 모델의 weight 파일을 로드합니다.

In [ ]:
with open(config_file_path, 'rb') as f_in:
    C = pickle.load(f_in)

class_mapping = C.class_mapping

if 'bg' not in class_mapping:
    class_mapping['bg'] = len(class_mapping)

class_mapping = {v: k for k, v in class_mapping.items()}
class_to_color = {class_mapping[v]: np.random.randint(0, 255, 3) for v in class_mapping}

In [ ]:
C.num_rois = 32

img_input = Input(shape=(None, None, 3))
roi_input = Input(shape=(C.num_rois, 4))
feature_map_input = Input(shape=(None, None, 1024))

shared_layers = nn.nn_base(img_input, trainable=True)
num_anchors = len(C.anchor_box_scales) * len(C.anchor_box_ratios)
rpn_layers = nn.rpn(shared_layers, num_anchors)
classifier = nn.classifier(feature_map_input, roi_input, C.num_rois, nb_classes=len(class_mapping), trainable=True)

model_rpn = Model(img_input, rpn_layers)
model_classifier_only = Model([feature_map_input, roi_input], classifier)
model_classifier = Model([feature_map_input, roi_input], classifier)

model_rpn.load_weights(weight_file_path, by_name=True)
model_classifier.load_weights(weight_file_path, by_name=True)

model_rpn.compile(optimizer='sgd', loss='mse')
model_classifier.compile(optimizer='sgd', loss='mse')

### 검출 시작

로드한 모델을 이용해서 소스 이미지의 객체 검출을 실행하고 출력합니다.

In [ ]:
def format_img_size(img, C):
    img_min_side = float(C.im_size)
    (height,width,_) = img.shape
    if width <= height:
        ratio = img_min_side/width
        new_height = int(ratio * height)
        new_width = int(img_min_side)
    else:
        ratio = img_min_side/height
        new_width = int(ratio * width)
        new_height = int(img_min_side)
    img = cv2.resize(img, (new_width, new_height), interpolation=cv2.INTER_CUBIC)
    return img, ratio

def format_img_channels(img, C):
    img = img[:, :, (2, 1, 0)]
    img = img.astype(np.float32)
    img[:, :, 0] -= C.img_channel_mean[0]
    img[:, :, 1] -= C.img_channel_mean[1]
    img[:, :, 2] -= C.img_channel_mean[2]
    img /= C.img_scaling_factor
    img = np.transpose(img, (2, 0, 1))
    img = np.expand_dims(img, axis=0)
    return img

def format_img(img, C):
    img, ratio = format_img_size(img, C)
    img = format_img_channels(img, C)
    return img, ratio

def get_real_coordinates(ratio, x1, y1, x2, y2):
    real_x1 = int(x1 / ratio)
    real_y1 = int(y1 / ratio)
    real_x2 = int(x2 / ratio)
    real_y2 = int(y2 / ratio)

    return (real_x1, real_y1, real_x2 ,real_y2)

In [ ]:
all_imgs = []
classes = {}

for dir_path, _, filenames in os.walk(predict_source):
    for filename in filenames:
        if not filename.endswith(('.jpg', '.png')):
            continue

        # 이미지 로드
        start = time.time()
        file_path = os.path.join(dir_path, filename)
        print(file_path)
        img = cv2.imread(file_path, cv2.IMREAD_COLOR)

        # 정규화 및 리사이징
        X, ratio = format_img(img, C)
        X = np.transpose(X, (0, 2, 3, 1))

        # 검출 (영역 제안)
        [Y1, Y2, F] = model_rpn.predict(X)
        R = roi_helpers.rpn_to_roi(Y1, Y2, C, K.image_dim_ordering(), overlap_thresh=0.7)
        R[:, 2] -= R[:, 0]
        R[:, 3] -= R[:, 1]

        # 검출 (영역 분류)
        bboxes = {}
        probs = {}
        
        for jk in range(R.shape[0]//C.num_rois + 1):
            ROIs = np.expand_dims(R[C.num_rois*jk:C.num_rois*(jk+1), :], axis=0)
            if ROIs.shape[1] == 0:
                break

            if jk == R.shape[0]//C.num_rois:
                curr_shape = ROIs.shape
                target_shape = (curr_shape[0],C.num_rois,curr_shape[2])
                ROIs_padded = np.zeros(target_shape).astype(ROIs.dtype)
                ROIs_padded[:, :curr_shape[1], :] = ROIs
                ROIs_padded[0, curr_shape[1]:, :] = ROIs[0, 0, :]
                ROIs = ROIs_padded

            [P_cls, P_regr] = model_classifier_only.predict([F, ROIs])

            for ii in range(P_cls.shape[1]):

                if np.max(P_cls[0, ii, :]) < bbox_threshold or np.argmax(P_cls[0, ii, :]) == (P_cls.shape[2] - 1):
                    continue

                cls_name = class_mapping[np.argmax(P_cls[0, ii, :])]

                if cls_name not in bboxes:
                    bboxes[cls_name] = []
                    probs[cls_name] = []

                (x, y, w, h) = ROIs[0, ii, :]

                cls_num = np.argmax(P_cls[0, ii, :])
                try:
                    (tx, ty, tw, th) = P_regr[0, ii, 4*cls_num:4*(cls_num+1)]
                    tx /= C.classifier_regr_std[0]
                    ty /= C.classifier_regr_std[1]
                    tw /= C.classifier_regr_std[2]
                    th /= C.classifier_regr_std[3]
                    x, y, w, h = roi_helpers.apply_regr(x, y, w, h, tx, ty, tw, th)
                except:
                    pass
                bboxes[cls_name].append([C.rpn_stride*x, C.rpn_stride*y, C.rpn_stride*(x+w), C.rpn_stride*(y+h)])
                probs[cls_name].append(np.max(P_cls[0, ii, :]))

        # 결과 후처리
        for key in bboxes:
            bbox = np.array(bboxes[key])

            new_boxes, new_probs = roi_helpers.non_max_suppression_fast(bbox, np.array(probs[key]), overlap_thresh=0.5)
            for jk in range(new_boxes.shape[0]):
                (x1, y1, x2, y2) = new_boxes[jk,:]

                (real_x1, real_y1, real_x2, real_y2) = get_real_coordinates(ratio, x1, y1, x2, y2)
                
                print('Label: {}, Score: {}, LTRB of the boundary box: {}, {}, {}, {}'.format(key, new_probs[jk], real_x1, real_y2, real_x2, real_y2))

                cv2.rectangle(img,(real_x1, real_y1), (real_x2, real_y2), (int(class_to_color[key][0]), int(class_to_color[key][1]), int(class_to_color[key][2])),2)

                textLabel = '{}: {}'.format(key,new_probs[jk])

                (retval,baseLine) = cv2.getTextSize(textLabel,cv2.FONT_HERSHEY_COMPLEX,1,1)
                textOrg = (real_x1, real_y1-0)

                cv2.rectangle(img, (textOrg[0] - 5, textOrg[1]+baseLine - 5), (textOrg[0]+retval[0] + 5, textOrg[1]-retval[1] - 5), (0, 0, 0), 2)
                cv2.rectangle(img, (textOrg[0] - 5,textOrg[1]+baseLine - 5), (textOrg[0]+retval[0] + 5, textOrg[1]-retval[1] - 5), (255, 255, 255), -1)
                cv2.putText(img, textLabel, textOrg, cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 0), 1)

        cv2.imwrite(os.path.join(predict_result, filename), img)

        print("Processing time: {}".format(time.time() - start))